In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

### all test

In [2]:
joint_2ds = {'train': [], 'test': []}
joint_2d_from_canonical_3ds = {'train': [], 'test': []}
confidences = {'train': [], 'test': []}
joint3d_images = {'train': [], 'test': []}
joint3d_image_from_canonical_3ds = {'train': [], 'test': []}
joints_25d_images = {'train': [], 'test': []}
_25d_factors = {'train': [], 'test': []}
camera_names = {'train': [], 'test': []}
actions = {'train': [], 'test': []}
sources = {'train': [], 'test': []}
frames = {'train': [], 'test': []}
world_3ds = {'train': [], 'test': []}
world_3d_from_canonical_3ds = {'train': [], 'test': []}
cam_3ds = {'train': [], 'test': []}
cam_3d_from_canonical_3ds = {'train': [], 'test': []}
cam_params = {'train': [], 'test': []}

# target 
pkl_folder = '/home/hrai/codes/MotionBERT/data/motion3d/fit3d_gt_pkl_canonical_3d_same_z'
test_subject = ['s04', 's05', 's07', 's08', 's09', 's10', 's11']
only_test = False

blacklist = []

for item in tqdm(os.listdir(pkl_folder)):
    splited = item.split('.')[0].split('_') 
    subject = splited[0]
    cam_num = splited[1]
    action = '_'.join(splited[2:])
    
    # filtering
    if only_test:
        if subject not in test_subject: continue
    if subject in blacklist: continue
    
    item_path = os.path.join(pkl_folder, item) 
    data = readpkl(item_path) 
    if subject in test_subject: data_type = 'test' 
    else: data_type = 'train' 

    joint_2ds[data_type] += list(np.array(data['joint_2d'])) 
    joint_2d_from_canonical_3ds[data_type] += list(np.array(data['joint_2d_from_canonical_3d'])) 
    confidences[data_type] += list(np.array(data['confidence'])) 
    joint3d_image_from_canonical_3ds[data_type] += list(np.array(data['joint3d_image_from_canonical_3d'])) 
    #joints_25d_images[data_type] += list(np.array(data['joints_2.5d_image_from_canonical_3d'])) 
    #_25d_factors[data_type] += data['2.5d_factor_from_canonical_3d'] 
    camera_names[data_type] += data['camera_name']  
    actions[data_type] += data['action'] 
    sources[data_type] += data['source'] 
    frames[data_type] += data['frame'] 
    world_3ds[data_type] += list(np.array(data['world_3d'])) 
    world_3d_from_canonical_3ds[data_type] += list(np.array(data['world_3d_from_canonical_3d']))
    cam_3ds[data_type] += list(np.array(data['cam_3d'])/1000.0) # mm to m 
    cam_3d_from_canonical_3ds[data_type] += list(np.array(data['cam_3d_from_canonical_3d'])/1000.0) 
    cam_params[data_type] += data['cam_param']
    
data_total = {'train': {}, 'test': {}}

for data_type in ['train', 'test']:
    data_total[data_type]['joint_2d']                            = np.array(joint_2ds[data_type])
    data_total[data_type]['joint_2d_from_canonical_3d']          = np.array(joint_2d_from_canonical_3ds[data_type])
    data_total[data_type]['confidence']                          = np.array(confidences[data_type])
    data_total[data_type]['joint3d_image_from_canonical_3d']     = np.array(joint3d_image_from_canonical_3ds[data_type])
    #data_total[data_type]['joints_2.5d_image_from_canonical_3d'] = copy.deepcopy(np.array(joints_25d_images[data_type]))
    #data_total[data_type]['2.5d_factor_from_canonical_3d']       = copy.deepcopy(np.array(_25d_factors[data_type]))
    data_total[data_type]['camera_name']                         = np.array(camera_names[data_type])
    data_total[data_type]['action']                              = np.array(actions[data_type])
    data_total[data_type]['source']                              = np.array(sources[data_type])
    data_total[data_type]['frame']                               = np.array(frames[data_type])
    data_total[data_type]['world_3d']                            = np.array(world_3ds[data_type])
    data_total[data_type]['world_3d_from_canonical_3d']          = np.array(world_3d_from_canonical_3ds[data_type])
    data_total[data_type]['cam_3d']                              = np.array(cam_3ds[data_type])
    data_total[data_type]['cam_3d_from_canonical_3d']            = np.array(cam_3d_from_canonical_3ds[data_type])
    data_total[data_type]['cam_param']                           = np.array(cam_params[data_type])
    
if len(data_total['train']['joint_2d']) == 0:
    for key in data_total['test'].keys():
        data_total['train'][key] = copy.deepcopy(data_total['test'][key][0:243])

data_total['train']['world_3d_from_canonical_3d'].shape, data_total['test']['world_3d_from_canonical_3d'].shape  

100%|██████████| 1504/1504 [01:08<00:00, 21.87it/s]


((230368, 17, 3), (1548924, 17, 3))

In [3]:
save_path = os.path.join('/home/hrai/codes/MotionBERT', f'data/motion3d/fit3d_gt_canonical_3d_same_z_tr_s03.pkl')
savepkl(data_total, save_path)

: 

In [1]:

frame_num = 24000
cam_3d = data_total['train']['cam_3d'][frame_num]
cam_3d_hat = get_rootrel_pose((cam_3d))
joint_2d = data_total['train']['joint_2d'][frame_num]
joint_2d_from_canonical_3d = data_total['train']['joint_2d_from_canonical_3d'][frame_num]
joint3d_image = data_total['train']['joint3d_image_from_canonical_3d'][frame_num]
#joints_25d_image = data_total['train']['joints_2.5d_image_from_canonical_3d'][frame_num]
#_25d_factor = data_total['train']['2.5d_factor_from_canonical_3d'][frame_num]

%matplotlib widget
fig = plt.figure(0)
fig.clear()
ax = axes_3d(fig, loc=121, view=(-90,-90), xlim=(0, 1000), ylim=(0, 1000))
ax2 = axes_2d(fig, loc=122)
#draw_3d_pose(ax, cam_3d_hat)
draw_3d_pose(ax, joint3d_image)
#draw_3d_pose(ax, joints_25d_image/_25d_factor, color='r')
draw_2d_pose(ax2, joint_2d)
draw_2d_pose(ax2, joint_2d_from_canonical_3d)
plt.show()

NameError: name 'data_total' is not defined